In [2]:
# Import OpenCV, tracker2 File, and Numpy
import cv2
from tracker2 import *
import numpy as np

# Initializations
end = 0
cap = cv2.VideoCapture("Resources/PINTU2USU.mp4")   # Video Source
f = 25                                              # Video frame per second
cf = 0                                              # Current Frame
predicted_dt = 7                                    # Predicted Distance Travel between two lines

# Tracker
tracker = EuclideanDistTracker()

# Object Detection
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40, detectShadows=True)

# KERNELS
kernelE = np.ones((5, 5), np.uint8)
kernelC = np.ones((15, 15), np.uint8)
kernelO = np.ones((3, 3), np.uint8)

# While the video is playing
while True:
    ret,frame = cap.read()
    cf = cap.get(cv2.CAP_PROP_POS_FRAMES)
    if not ret:
        break

    # frame = cv2.resize(frame, None, fx=0.5, fy=0.5)
    height,width,_ = frame.shape

    # Extract Region Of Interest
    # frame = frame[0:288,0:352]

    # Initial Frame w/ Object detection
    mask = object_detector.apply(frame)

    # Pre-processing Frame
    mask1 = cv2.GaussianBlur(mask, (5, 5), 0)
    mask1 = cv2.dilate(mask1, None)

    # Get Only White (remove shadows)
    _, mask1 = cv2.threshold(mask1, 240, 255, cv2.THRESH_BINARY)
    
    # Morphology
    # MORPH_OPEN
    # mask1 = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernelO)
    
    # MORPH_CLOSE
    mask2 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernelC)

    # Erosion
    e_img = cv2.erode(mask2, None)

    # Contours
    contours,_ = cv2.findContours(e_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    detections = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        # THRESHOLD
        if area > 400:
            cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
        if area > 1500:
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)

            detections.append([x,y,w,h])

    # Object Tracking (in tracker2.py)
    boxes_ids = tracker.update(detections, cf)
    for box_id in boxes_ids:
        x,y,w,h,id = box_id

        # Get Speed by id
        s = tracker.getsp(id, predicted_dt, f)

        # If exceed Speed Limit
        if(s < tracker.limit()):
            cv2.putText(frame,str(id)+" "+str(s), (x,y-15), cv2.FONT_HERSHEY_PLAIN,1,(255,255,0),2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
        else:
            cv2.putText(frame,str(id)+ " "+str(s),(x, y-15),cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255),2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 165, 255), 3)

        cv2.circle(frame, tracker.center_points[id], 2, (255, 255, 0), 2)
        # Capture vehicle when already passed with its speed
        if (tracker.f[id] == 1 and s != 0):
            tracker.capture(frame, x, y, h, w, s, id)
            print(tracker.s1[0, id], tracker.s2[0, id])

    # DRAW LINES
    cv2.line(frame, (0, 440), (width, 440), (0, 0, 255), 2)
    cv2.line(frame, (0, 460), (width, 460), (0, 0, 255), 2)

    cv2.line(frame, (0, 320), (width, 320), (0, 0, 255), 2)
    cv2.line(frame, (0, 340), (width, 340), (0, 0, 255), 2)

    cv2.putText(frame,str(cf),(100, 100),cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255),2)

    #DISPLAY
    cv2.imshow("Mask",mask)
    cv2.imshow("Pre-process",mask1)
    cv2.imshow("Post-process",mask2)
    cv2.imshow("Erode", e_img)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(0)     # Hold to play
    #key = cv2.waitKey(f)    # Autoplay and match FPS
    # ESC Key to Stop Program
    if key==27:
        tracker.end()
        end=1
        break

if(end!=1):
    tracker.end()

cap.release()
cv2.destroyAllWindows()

155.0 179.0
155.0 180.0
155.0 181.0
155.0 182.0
155.0 183.0
155.0 184.0
155.0 185.0
155.0 186.0
155.0 186.0
254.0 278.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
254.0 279.0
308.0 331.0
308.0 334.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
308.0 335.0
342.0 363.0
342.0 364.0
342.0 365.0
342.0 366.0
342.0 367.0
550.0 575.0
550.0 576.0
550.0 577.0
550.0 577.0
550.0 577.0
550.0 577.0
562.0 580.0
550.0 577.0
562.0 581.0
550.0 577.0
562.0 582.0
550.0 577.0
562.0 583.0
550.0 577.0
562.0 584.0
550.0 577.0
550.0 577.0
550.0 577.0
550.0 577.0
550.0 577.0
550.0 577.0
550.0 577.0
550.0 577.0
550.0 577.0
596.0 616.0
596.0 617.0
596.0 617.0
596.0 617.0
596.0 617.0
596.